In [26]:
from agents.rag_agent import get_agent


In [27]:
import pandas as pd
df = pd.read_csv("/Users/admin/Working/nckh-chatbot-neo4j/chatbot_api/src/test_fit241222-01.csv")
# df = df.head(5)
questions = df['user_input']
ground_truth = df['reference']
retrieval_context = df['reference_contexts']
questions

0     Học viện đào tạo công nghệ IT Plus có vai trò ...
1     Khoa CNTT của Đại học Công nghiệp Hà Nội đã đạ...
2     Học viện đào tạo công nghệ IT Plus có vai trò ...
3     PGS.TS. Phạm Văn Bổng đã có những phát biểu gì...
4     Ban Giám hiệu Nhà trường đã có những đóng góp ...
                            ...                        
57    Hội thảo khoa học về AI và ứng dụng trong đào ...
58    Trường Đại học Công nghiệp Hà Nội có những cơ ...
59    Trường Đại học Công nghiệp Hà Nội đã tổ chức n...
60    Trường Đại học Công nghiệp Hà Nội đã tổ chức n...
61    Trường Đại học Công nghiệp Hà Nội đã tổ chức n...
Name: user_input, Length: 62, dtype: object

In [28]:
from chains.semantic_search_chunk_chain import get_chunk_retriever
import uuid
model_name = "phobert"
retriever = get_chunk_retriever(model_name)
print(retriever)

tags=['Neo4jVector', 'HuggingFaceEmbeddings'] vectorstore=<langchain_neo4j.vectorstores.neo4j_vector.Neo4jVector object at 0x16dd0ab50> search_type='similarity_score_threshold' search_kwargs={'score_threshold': 0.7, 'k': 2}


In [29]:
retriever.invoke("Ngành đào tạo tại trường đại học công nghiệp ")

[Document(metadata={'link': ['https://www.haui.edu.vn/vn/tin-tuc/dai-hoc-cong-nghiep-ha-noi-du-kien-mot-so-diem-moi-trong-tuyen-sinh-dai-hoc-chinh-quy-nam-2024/64658'], 'filename': ['Đại học Công Nghiệp Hà Nội dự kiến một số điểm mới trong tuyển sinh đại học chính quy năm 2024']}, page_content='content: đủ điều kiện tốt nghiệp được cấp 2 bằng của Đại học Công nghiệp Hà Nội và Đại học Khoa học Kỹ thuật Quảng Tây; + Sinh viên trúng tuyển nhập học ngành Quản trị kinh doanh, ngành Marketing, ngành Quản trị khách sạn có thể đăng ký theo chương trình liên kết đào tạo 2+2 giữa Đại học Công nghiệp Hà Nội với Đại học Bách khoa Quế Lâm - Trung Quốc, học 2 năm tại Đại học Công nghiệp Hà Nội và 2 năm tại Đại học Bách khoa Quế Lâm. Sinh viên hoàn thành chương trình đào tạo và đủ điều kiện tốt nghiệp được cấp 2 bằng của Đại học Công nghiệp Hà Nội và Đại học Bách khoa Quế Lâm. 3. Học bổng Năm 2024, Đại học Công nghiệp Hà Nội dự kiến cấp học bổng cho sinh viên có kết quả xét tuyển đầu vào xuất sắc với

In [30]:
import time

answers = []
contexts = []
retriever_times = []
invoke_times = []

for query in questions:
    # Measure time for retriever
    start_retriever = time.time()
    retrieved_docs = retriever.invoke(query)
    retriever_time = time.time() - start_retriever
    retriever_times.append(retriever_time)
    
    # Collect context from retriever
    contexts.append([docs.page_content for docs in retrieved_docs])
    
    # Measure time for chat_agent.invoke
    start_invoke = time.time()
    answer = get_agent().invoke(
        {"input": query}).get("output", "No output text provided.")
    invoke_time = time.time() - start_invoke
    invoke_times.append(invoke_time)
    
    # Collect answer
    answers.append(answer)

# Print time statistics



> Entering new AgentExecutor chain...

Invoking: `get_chunk_tool` with `{'question': 'Học viện đào tạo công nghệ IT Plus có vai trò gì trong lễ kỷ niệm 20 năm thành lập khoa CNTT?'}`


[Document(metadata={'link': ['https://www.haui.edu.vn/vn/bao-chi-viet-ve-haui/giaoduc-haui-co-them-nhieu-ctdt-dat-chuan-kiem-dinh-chat-luong-quoc-gia-va-quoc-te/65696'], 'filename': ['[giaoduc] HaUI có thêm nhiều CTĐT đạt chuẩn kiểm định chất lượng quốc gia và quốc tế']}, page_content='content: [giaoduc] HaUI có thêm nhiều CTĐT đạt chuẩn kiểm định chất lượng quốc gia và quốc tế Thứ Bảy, 16:08 16/11/2024 63 GDVN-Năm học vừa qua, Trường ĐH Công nghiệp Hà Nội có thêm 15 chương trình đào tạo đạt chuẩn quốc gia và 5 chương trình đạt chuẩn kiểm định ABET. Ngày 16/11, Trường Đại học Công nghiệp Hà Nội tổ chức Lễ kỷ niệm 42 năm Ngày Nhà giáo Việt Nam, công bố và trao chứng nhận kiểm định chất lượng cho 6 chương trình đào tạo đạt chuẩn quốc gia; công bố các chương trình đào tạo đạt chuẩn quốc tế ABET. Toàn cả

In [23]:
import datetime
import csv
import os


df = df.head(2)
# Create a dynamic filename
results_dir = os.path.join("results", model_name)
os.makedirs(results_dir, exist_ok=True)


# Get the current date and time
current_datetime = datetime.datetime.now()

# Format the current date and time
formatted_datetime = current_datetime.strftime("%y%m%d-%H%M")

filename = f"response_fit{formatted_datetime}_1"
file_path = os.path.join(results_dir, filename)

print(filename)

df["response"] = answers
df["retrieved_contexts"] = contexts
# df["retriever_time"] = retriever_times
# df["invoke_time"] = invoke_times
df.to_csv(
    f"{file_path}.csv",
    index=False,  # Exclude the DataFrame index from the CSV
    quoting=csv.QUOTE_ALL,  # Enclose all fields in double quotes
    escapechar='\\',  # Escape character for special characters (optional)
    encoding='utf-8'  # Ensure proper encoding for special characters
)


response_fit241222-1228_1


In [24]:
from ragas.dataset_schema import EvaluationDataset
from datasets import Dataset
import pandas as pd
import ast
from llm.get_llm import get_eval_model_function

# Read the CSV file
df = pd.read_csv(f"{file_path}.csv")
# df = df.head(2)

# Define a safe evaluation function
def safe_literal_eval(val):
    try:
        # Try to evaluate the value as a Python literal
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        # Return an empty list if evaluation fails
        return []

# Apply the safe evaluation function to the columns
df['retrieved_contexts'] = df['retrieved_contexts'].astype(str).apply(safe_literal_eval)
df['reference_contexts'] = df['reference_contexts'].astype(str).apply(safe_literal_eval)

# Process the DataFrame in chunks of 30 rows
chunk_size = 5

for start_idx in range(0, len(df), chunk_size):
    # Slice the DataFrame to get the current chunk
    df_chunk = df.iloc[start_idx:start_idx + chunk_size]
    
    # Convert the chunk to a Hugging Face Dataset
    dataset = Dataset.from_pandas(df_chunk)

    from ragas import evaluate
    from ragas.cost import get_token_usage_for_openai
    from ragas.metrics import (
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
        answer_similarity,
        answer_correctness,
    )

    # Evaluate the dataset
    result = evaluate(
        dataset=dataset,
        llm=get_eval_model_function(),
        # token_usage_parser=get_token_usage_for_openai,
        
    )
    # print(result.total_tokens())


    # Convert results to a DataFrame
    df_result = result.to_pandas()
    
    # Add extra columns (adjust indexing for time arrays if necessary)
    df_result["retriever_time"] = retriever_times[start_idx:start_idx + chunk_size]
    df_result["invoke_time"] = invoke_times[start_idx:start_idx + chunk_size]

    # Append results to the output file
    df_result.to_csv(
        f"{file_path}_result.csv",
        mode='a',  # Append mode
        header=(start_idx == 0),  # Write the header only for the first chunk
        index=False
    )

get llm 


Evaluating: 100%|██████████| 8/8 [00:26<00:00,  3.31s/it]


In [25]:
df.describe()

,user_input,reference_contexts,reference,response,retrieved_contexts
count,2,2,2,2,2
unique,2,1,2,2,1
top,Học viện đào tạo công nghệ IT Plus có vai trò ...,[Lễ kỉ niệm 20 năm thành lập khoa CNTT Sáng ng...,Học viện đào tạo công nghệ IT Plus là một tron...,Tôi không có thông tin này và khuyến nghị bạn ...,[content: Lễ kỉ niệm 20 năm thành lập khoa CNT...
freq,1,2,1,1,2


In [ ]:
result

In [ ]:
print(result.total_tokens())


In [61]:
# df_result["retriever_time"] = retriever_times[30:]
# df_result["invoke_time"] = invoke_times[30:]
# df_result.to_csv(f"second_result_{filename}")